In [58]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import copy

def Lgb_model(x_train, x_valid, y_train, y_valid,x_test,category=None):#(X_train,Y_train,X_test,category=None):

    params = {
        'task':'train',
        'application':'regression',
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': {'auc'},
        'num_leaves': 31,
        'min_sum_hessian_in_leaf':1e-3,
        'min_data_in_leaf':20,
        #'num_iterations':100,
        'learning_rate': 0.1,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.9,
        #'bagging_freq': 5,
        'verbose': 0
    }
    #print(category)
    loss=dict()
    lgb_train = lgb.Dataset(x_train, y_train,categorical_feature=category)
    lgb_eval = lgb.Dataset(x_valid, y_valid,categorical_feature=category)
    # lgb_eval = lgb.Dataset(x_valid, y_valid, reference=lgb_train)
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=10000,
    #                 valid_sets=lgb_eval,
                    valid_sets=[lgb_train,lgb_eval],
                    valid_names=['Train', 'Valid'],
                    evals_result=loss,
                    categorical_feature=category,
                    early_stopping_rounds=200,
                   )
    
    booster = gbm
    importance = booster.feature_importance(importance_type='split')
    feature_name = booster.feature_name()
    # for (feature_name,importance) in zip(feature_name,importance):
    #     print (feature_name,importance) 
    feature_importance = pd.DataFrame({'feature_name':feature_name,'importance':importance} )

    
    
    y_pred=gbm.predict(x_test, num_iteration=gbm.best_iteration)
    

    return [y_pred],feature_importance,max(loss['Valid']['auc'])#返回预测结果，特征重要性和验证集的auc


In [59]:
dataPath='./mid_dir/'
del_list=['企业名称','flag']
drop_feature1=[]
drop_feature2=[]


trainset=pd.read_csv(dataPath+'trainset.csv')
testset=pd.read_csv(dataPath+'testset.csv')

print('----------------------阶段4：模型训练阶段-------------------------')
train_data=trainset.drop(del_list,axis=1).drop(['L1','L2'],axis=1)
test_data=testset.drop(del_list,axis=1)


#[x for x in list(trainset.columns)if x not in del_list]
category_list=['种类_x','实收资金','企业类型代码','所属行业代码',
             '发证机关名称', '行政区划','是否有经纬度','法定代表人证件名称',
             '登记注册类型_x','审核结果', '审核单位', '纳税人状态','登记注册类型_y','管理机构','数据来源','审批类别',
            '地方编码','种类_y', '许可证编号',
             '许可证名称','注（撤、吊）销原因', '注（撤、吊）销批准机关全称',
             '权力编码', '年检年度','年检结果',
             '年检事项名称','种类', '资质名称','认定机关全称','dp1','dp2', 'dp3', 'dp4', 'dp5', 'dp6', 'dp7', 'dp8',
            '税务管理码_x', '税务管理码_y','统一社会信用代码','法定代表人姓名','纳税人识别号_y','纳税人识别号_x','行政许可编码','法定代表人名称',
             '行政相对人名称','被处罚的自然人身份证号','被处罚的企业法定代表人身份证号' ,
              
             '是否有许可' ,'是否被罚','是否注销','是否年检','有无资质','有无招聘信息','是否表彰',
              ]


category_list1=[x for x in category_list if x not in drop_feature1]
category_list2=[x for x in category_list if x not in drop_feature2]

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5,shuffle=True)
result=np.array([0 for i in range(len(test_data))])
result=result.astype(np.float64)
result=result.reshape(1,len(test_data))

feature_importance1=[]
feature_importance2=[]
Loss1=[]
Loss2=[]

pre1=copy.deepcopy(result)

for train_index, valid_index in skf.split(train_data,trainset['L1']):
    re,t,loss=Lgb_model(train_data.drop(drop_feature1,axis=1).iloc[train_index],
                   train_data.drop(drop_feature1,axis=1).iloc[valid_index],
                   trainset.iloc[train_index]['L1'],
                   trainset.iloc[valid_index]['L1'],
                   test_data.drop(drop_feature1,axis=1),category=category_list1)
    pre1+=re
    feature_importance1.append(t)
    Loss1.append(loss)
pre1=pre1/5
pre1
print('\n\n\n\n\n\n\n')
print('fdfdfd',result)

# pre2=copy.deepcopy(result)
# for train_index, valid_index in skf.split(train_data,trainset['L2']):
#     re,t,loss=Lgb_model(train_data.drop(drop_feature2,axis=1).iloc[train_index],
#                    train_data.drop(drop_feature2,axis=1).iloc[valid_index],
#                    trainset.iloc[train_index]['L2'],
#                    trainset.iloc[valid_index]['L2'],
#                    test_data.drop(drop_feature2,axis=1),category=category_list2)
#     pre2+=re
#     feature_importance2.append(t)
#     Loss2.append(loss)
# pre2=pre2/5
# pre2
# print('----------------------阶段4结束-------------------------------------')

----------------------阶段4：模型训练阶段-------------------------


c:\users\mr.handsome\appdata\local\programs\python\python36\lib\site-packages\lightgbm\basic.py:685: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[1]	Train's auc: 0.855318	Valid's auc: 0.825796
Training until validation scores don't improve for 200 rounds.
[2]	Train's auc: 0.88182	Valid's auc: 0.854694
[3]	Train's auc: 0.912167	Valid's auc: 0.877043
[4]	Train's auc: 0.925634	Valid's auc: 0.886578
[5]	Train's auc: 0.930128	Valid's auc: 0.890304
[6]	Train's auc: 0.934589	Valid's auc: 0.89304
[7]	Train's auc: 0.938593	Valid's auc: 0.896378
[8]	Train's auc: 0.943676	Valid's auc: 0.898703
[9]	Train's auc: 0.947172	Valid's auc: 0.902802
[10]	Train's auc: 0.949213	Valid's auc: 0.905946
[11]	Train's auc: 0.95082	Valid's auc: 0.907791
[12]	Train's auc: 0.952046	Valid's auc: 0.908087
[13]	Train's auc: 0.953692	Valid's auc: 0.908655
[14]	Train's auc: 0.954878	Valid's auc: 0.910174
[15]	Train's auc: 0.956891	Valid's auc: 0.910949
[16]	Train's auc: 0.959046	Valid's auc: 0.911414
[17]	Train's auc: 0.960684	Valid's auc: 0.91101
[18]	Train's auc: 0.962202	Valid's auc: 0.910711
[19]	Train's auc: 0.963253	Valid's auc: 0.910786
[20]	Train's auc: 0

[171]	Train's auc: 0.997477	Valid's auc: 0.906669
[172]	Train's auc: 0.997503	Valid's auc: 0.906626
[173]	Train's auc: 0.997529	Valid's auc: 0.906709
[174]	Train's auc: 0.997571	Valid's auc: 0.906844
[175]	Train's auc: 0.997588	Valid's auc: 0.906718
[176]	Train's auc: 0.997594	Valid's auc: 0.906661
[177]	Train's auc: 0.997616	Valid's auc: 0.906513
[178]	Train's auc: 0.997626	Valid's auc: 0.906294
[179]	Train's auc: 0.997631	Valid's auc: 0.906186
[180]	Train's auc: 0.997662	Valid's auc: 0.906088
[181]	Train's auc: 0.997674	Valid's auc: 0.905915
[182]	Train's auc: 0.997698	Valid's auc: 0.905723
[183]	Train's auc: 0.99772	Valid's auc: 0.905626
[184]	Train's auc: 0.997733	Valid's auc: 0.905268
[185]	Train's auc: 0.997745	Valid's auc: 0.905245
[186]	Train's auc: 0.997753	Valid's auc: 0.904449
[187]	Train's auc: 0.99777	Valid's auc: 0.904454
[188]	Train's auc: 0.9978	Valid's auc: 0.904203
[189]	Train's auc: 0.997816	Valid's auc: 0.904219
[190]	Train's auc: 0.997839	Valid's auc: 0.9042
[191]	

[88]	Train's auc: 0.991155	Valid's auc: 0.924557
[89]	Train's auc: 0.991207	Valid's auc: 0.924351
[90]	Train's auc: 0.991339	Valid's auc: 0.924373
[91]	Train's auc: 0.99143	Valid's auc: 0.924345
[92]	Train's auc: 0.991613	Valid's auc: 0.924245
[93]	Train's auc: 0.991739	Valid's auc: 0.924422
[94]	Train's auc: 0.991822	Valid's auc: 0.92419
[95]	Train's auc: 0.992012	Valid's auc: 0.924378
[96]	Train's auc: 0.992062	Valid's auc: 0.924626
[97]	Train's auc: 0.992108	Valid's auc: 0.924432
[98]	Train's auc: 0.99224	Valid's auc: 0.924452
[99]	Train's auc: 0.992358	Valid's auc: 0.924547
[100]	Train's auc: 0.992448	Valid's auc: 0.924448
[101]	Train's auc: 0.992532	Valid's auc: 0.924424
[102]	Train's auc: 0.99266	Valid's auc: 0.924466
[103]	Train's auc: 0.99282	Valid's auc: 0.924541
[104]	Train's auc: 0.99296	Valid's auc: 0.924588
[105]	Train's auc: 0.993068	Valid's auc: 0.924589
[106]	Train's auc: 0.993129	Valid's auc: 0.924575
[107]	Train's auc: 0.993208	Valid's auc: 0.924417
[108]	Train's auc:

...

[172]	Train's auc: 0.997306	Valid's auc: 0.891874
[173]	Train's auc: 0.99732	Valid's auc: 0.891641
[174]	Train's auc: 0.997337	Valid's auc: 0.891784
[175]	Train's auc: 0.997442	Valid's auc: 0.891443
[176]	Train's auc: 0.997453	Valid's auc: 0.891562
[177]	Train's auc: 0.997465	Valid's auc: 0.891683
[178]	Train's auc: 0.99748	Valid's auc: 0.891688
[179]	Train's auc: 0.997487	Valid's auc: 0.891898
[180]	Train's auc: 0.99752	Valid's auc: 0.891865
[181]	Train's auc: 0.997539	Valid's auc: 0.891856
[182]	Train's auc: 0.997555	Valid's auc: 0.891523
[183]	Train's auc: 0.997568	Valid's auc: 0.891378
[184]	Train's auc: 0.997608	Valid's auc: 0.891306
[185]	Train's auc: 0.997619	Valid's auc: 0.891224
[186]	Train's auc: 0.997633	Valid's auc: 0.891076
[187]	Train's auc: 0.997657	Valid's auc: 0.890821
[188]	Train's auc: 0.997671	Valid's auc: 0.890607
[189]	Train's auc: 0.997683	Valid's auc: 0.890565
[190]	Train's auc: 0.997695	Valid's auc: 0.890814
[191]	Train's auc: 0.997708	Valid's auc: 0.890952
[19

[76]	Train's auc: 0.990014	Valid's auc: 0.912953
[77]	Train's auc: 0.990365	Valid's auc: 0.912465
[78]	Train's auc: 0.990461	Valid's auc: 0.912522
[79]	Train's auc: 0.990571	Valid's auc: 0.912155
[80]	Train's auc: 0.990876	Valid's auc: 0.911693
[81]	Train's auc: 0.991074	Valid's auc: 0.911676
[82]	Train's auc: 0.991156	Valid's auc: 0.911816
[83]	Train's auc: 0.99129	Valid's auc: 0.91201
[84]	Train's auc: 0.991367	Valid's auc: 0.912177
[85]	Train's auc: 0.991465	Valid's auc: 0.912038
[86]	Train's auc: 0.991597	Valid's auc: 0.912042
[87]	Train's auc: 0.991806	Valid's auc: 0.911538
[88]	Train's auc: 0.991879	Valid's auc: 0.911303
[89]	Train's auc: 0.992052	Valid's auc: 0.911111
[90]	Train's auc: 0.992128	Valid's auc: 0.910756
[91]	Train's auc: 0.992197	Valid's auc: 0.91077
[92]	Train's auc: 0.992288	Valid's auc: 0.910547
[93]	Train's auc: 0.992394	Valid's auc: 0.910609
[94]	Train's auc: 0.99247	Valid's auc: 0.910446
[95]	Train's auc: 0.992517	Valid's auc: 0.910639
[96]	Train's auc: 0.9925

[243]	Train's auc: 0.998563	Valid's auc: 0.906682
[244]	Train's auc: 0.998577	Valid's auc: 0.906692
[245]	Train's auc: 0.998593	Valid's auc: 0.906662
[246]	Train's auc: 0.998601	Valid's auc: 0.906612
[247]	Train's auc: 0.998615	Valid's auc: 0.90659
Early stopping, best iteration is:
[47]	Train's auc: 0.982628	Valid's auc: 0.914001
[1]	Train's auc: 0.828245	Valid's auc: 0.746215
Training until validation scores don't improve for 200 rounds.
[2]	Train's auc: 0.889164	Valid's auc: 0.835882
[3]	Train's auc: 0.915071	Valid's auc: 0.873812
[4]	Train's auc: 0.929699	Valid's auc: 0.884898
[5]	Train's auc: 0.932844	Valid's auc: 0.886266
[6]	Train's auc: 0.937379	Valid's auc: 0.891357
[7]	Train's auc: 0.940482	Valid's auc: 0.89265
[8]	Train's auc: 0.942796	Valid's auc: 0.895574
[9]	Train's auc: 0.94597	Valid's auc: 0.896089
[10]	Train's auc: 0.948085	Valid's auc: 0.897429
[11]	Train's auc: 0.949797	Valid's auc: 0.897359
[12]	Train's auc: 0.951708	Valid's auc: 0.897492
[13]	Train's auc: 0.953533	

[160]	Train's auc: 0.996814	Valid's auc: 0.909704
[161]	Train's auc: 0.996833	Valid's auc: 0.909569
[162]	Train's auc: 0.996872	Valid's auc: 0.909524
[163]	Train's auc: 0.996914	Valid's auc: 0.90936
[164]	Train's auc: 0.996982	Valid's auc: 0.909349
[165]	Train's auc: 0.997013	Valid's auc: 0.909318
[166]	Train's auc: 0.997064	Valid's auc: 0.90939
[167]	Train's auc: 0.99708	Valid's auc: 0.908876
[168]	Train's auc: 0.997139	Valid's auc: 0.908931
[169]	Train's auc: 0.997184	Valid's auc: 0.908737
[170]	Train's auc: 0.99721	Valid's auc: 0.908742
[171]	Train's auc: 0.997216	Valid's auc: 0.908888
[172]	Train's auc: 0.997251	Valid's auc: 0.908634
[173]	Train's auc: 0.997271	Valid's auc: 0.908728
[174]	Train's auc: 0.997303	Valid's auc: 0.9085
[175]	Train's auc: 0.997338	Valid's auc: 0.908486
[176]	Train's auc: 0.997354	Valid's auc: 0.908461
[177]	Train's auc: 0.997409	Valid's auc: 0.90868
[178]	Train's auc: 0.997447	Valid's auc: 0.908704
[179]	Train's auc: 0.997477	Valid's auc: 0.90881
[180]	Tr

[330]	Train's auc: 0.99939	Valid's auc: 0.906548
[331]	Train's auc: 0.999395	Valid's auc: 0.906692
[332]	Train's auc: 0.999396	Valid's auc: 0.906717
[333]	Train's auc: 0.999399	Valid's auc: 0.906865
[334]	Train's auc: 0.999408	Valid's auc: 0.906767
[335]	Train's auc: 0.99941	Valid's auc: 0.906776
[336]	Train's auc: 0.999415	Valid's auc: 0.906978
[337]	Train's auc: 0.999425	Valid's auc: 0.907171
[338]	Train's auc: 0.999427	Valid's auc: 0.907013
[339]	Train's auc: 0.99943	Valid's auc: 0.906871
[340]	Train's auc: 0.999431	Valid's auc: 0.906774
[341]	Train's auc: 0.999431	Valid's auc: 0.906823
[342]	Train's auc: 0.999433	Valid's auc: 0.906861
[343]	Train's auc: 0.999438	Valid's auc: 0.906952
[344]	Train's auc: 0.99944	Valid's auc: 0.907095
[345]	Train's auc: 0.999444	Valid's auc: 0.907179
[346]	Train's auc: 0.999445	Valid's auc: 0.906999
[347]	Train's auc: 0.999452	Valid's auc: 0.9071
[348]	Train's auc: 0.999466	Valid's auc: 0.907157
[349]	Train's auc: 0.999471	Valid's auc: 0.907222
[350]	

In [23]:
fi_list1=pd.concat([x for x in feature_importance1])
fi_list1=fi_list1['importance'].groupby(fi_list1['feature_name']).mean().reset_index().sort_values(by=['importance'],ascending=False)

fi_list2=pd.concat([x for x in feature_importance2])
fi_list2=fi_list2['importance'].groupby(fi_list2['feature_name']).mean().reset_index().sort_values(by=['importance'],ascending=False)


[0.9133677058720998,
 0.926668999939132,
 0.8952556455048999,
 0.9140014608314566,
 0.9103416579159628]

In [26]:
fi_list1

,feature_name,importance
47,注册（开办）资金,517.4
29,所属行业代码,384.4
73,许可项数,174.8
30,投资人数,160.2
17,发证机关名称,158.8
9,出资比例,145.6
27,年检次数,133.0
77,高管人数,121.2
54,种类_x,111.8
75,资质项数,54.8


In [15]:
submit=DataFrame(testset['企业名称'].values,columns=['EID'])
submit
submit['FORTARGET1']=pre1[0]
submit['FORTARGET2']=pre2[0]

submit.to_csv(input_path+'compliance_assessment.csv',encoding='gbk',index=False)

,feature_name,importance
61,纳税人识别号_y,0.0
71,许可证名称,0.0
39,是否被罚,0.0
40,有无招聘信息,0.0
57,税务管理码_y,0.0
60,纳税人识别号_x,0.0
7,dp8,0.0
68,被处罚的自然人身份证号,0.0
49,注（撤、吊）销原因,0.0
53,种类,0.0
